In [1]:
import pandas as pd
import numpy as np

In [2]:
def start_pipeline(dataf):
    
    return dataf.copy()

def select_initial_columns(dataf,
                           columns=[
                                'dealnumber',
                                'dealstatus',
                                'withdrawn_date',
                                'completed_date',
                                'target_sector',
                                'acquiror_sector',
                                'deal_type',
                                'bid_premium_rumour_date',
                                'bid_premium_announced_date',
                                'initial_stake',
                                'acquiror_country',
                                'target_country',
                                'predeal_target_market_cap',
                                'predeal_acquiror_market_cap',
                                'predeal_acquiror_net_assets',
                                'predeal_target_total_assets',
                                'predeal_target_ebit',
                                'target_name',
                                'acquiror_name',
                           ]):
    
    dataf = dataf.drop(columns=['Unnamed: 0'])
    dataf.columns = columns
    return dataf

def select_initial_rows(dataf):
    
    dataf = dataf.loc[lambda x: ~x['dealnumber'].isnull()].copy()
    dataf['dealnumber'] = pd.factorize(dataf['dealnumber'])[0] + 1
    return dataf

def delete_na(dataf):

    for c in dataf.columns:
        try:
            dataf[c] = dataf[c].str.replace('n.a.', np.nan)
        except:
            pass
        try:
            dataf[c] = dataf[c].replace('n.a.', np.nan)
        except:
            pass   
    return dataf

def feature_generation(dataf):
    
    dataf['target_sector'] = dataf['target_sector'].str.split(',').str[0]
    dataf['acquiror_sector'] = dataf['acquiror_sector'].str.split(',').str[0]
    dataf['same_sector'] = False
    dataf.loc[lambda x: x['target_sector']==x['acquiror_sector'], 'same_sector'] = True
    
    dataf['same_country'] = False
    dataf.loc[lambda x: x['target_country']==x['acquiror_country'], 'same_country'] = True
    
    dataf['relative_size'] = dataf['predeal_target_market_cap'] / dataf['predeal_acquiror_market_cap']
    
    dataf['roa_target'] = round(dataf['predeal_target_ebit'] / dataf['predeal_target_total_assets'], 6)
    
    dataf['acquiror_size'] = dataf['predeal_acquiror_market_cap']
    dataf['bid_premium'] = dataf['bid_premium_rumour_date']
    dataf['acquiror_free_cashflow'] = dataf['predeal_acquiror_net_assets']
    
    return dataf

def feature_selection(dataf,
                      columns=[
                          'dealstatus',
                          'same_sector',
                          'same_country',
                          'target_sector',
                          'acquiror_sector',
                          'deal_type',
                          'relative_size',
                          'acquiror_size',
                          'roa_target',
                          'bid_premium',
                          'initial_stake',
                          'acquiror_free_cashflow',
                      ]):
    
    return dataf[columns]

def report_empty_rows(dataf):
    
    for c in dataf.columns:
        print(f'{c}: {round(len(dataf.loc[lambda x: ~x[c].isnull()]) / len(dataf), 2)}')

def filter_out_empty_rows(dataf, columns=None):
    
    if columns == None:
        for c in dataf.columns:
            dataf = dataf.loc[lambda x: ~x[c].isnull()]
            dataf = dataf.loc[lambda x: x[c]!=-np.inf]
            dataf = dataf.loc[lambda x: x[c]!=-np.inf] 
    else:
        for c in columns:
            dataf = dataf.loc[lambda x: ~x[c].isnull()]
            dataf = dataf.loc[lambda x: x[c]!=-np.inf]
            dataf = dataf.loc[lambda x: x[c]!=-np.inf] 
    return dataf

def select_columns(dataf, columns):
    
    return dataf[columns]

def clean_sector(dataf):
    
    for c in ['target_sector', 'acquiror_sector']:
        dataf[c] = dataf[c].str.split(' ').str[0]
    return dataf

def transform_low_frequent_sectors(dataf, new_value):
    
    for c in ['target_sector', 'acquiror_sector']:
        dataf.loc[lambda x: x.groupby([c])['dealstatus'].transform('count')<10, c] = new_value
    return dataf

def remove_roa_outliers(dataf):
    
    highest = dataf['roa_target'].quantile(0.97)
    lowest = dataf['roa_target'].quantile(0.03)
    return dataf.loc[lambda x: (x['roa_target']>lowest) & (x['roa_target']<highest)]

def define_dummies(dataf, columns):
    
    for c in columns:
        for u in dataf[c].unique():
            dummy_col_name = f"{c}_{u.lower().replace(' ', '_')}"
            dataf[dummy_col_name] = False
            dataf.loc[lambda x: x[c]==u, dummy_col_name] = True
            
    return dataf.drop(columns=columns)

def clean_dealtype(dataf):
    
    dataf['deal_type'] = dataf['deal_type'].str.lower()
    dataf.loc[lambda x: x['deal_type'].str.contains('acquisition increased'), 'deal_type_clean'] = 'acquisition_stake_increased'
    dataf.loc[lambda x: (x['deal_type'].str.contains('100%'))
                      & (x['deal_type'].str.contains('acquisition')), 'deal_type_clean'] = 'acquisition_100%'
    dataf.loc[lambda x: (x['deal_type'].str.contains('acquisition'))
                      & (x['deal_type_clean'].isnull()), 'deal_type_clean'] = 'acquisition_other%'
    dataf.loc[lambda x: (x['deal_type'].str.contains('merger'))
                      & (x['deal_type_clean'].isnull()), 'deal_type_clean'] = 'merger'
    dataf.loc[lambda x: x['deal_type_clean'].isnull(), 'deal_type_clean'] = 'other'
    dataf['deal_type'] = dataf['deal_type_clean']
    
    return dataf.drop(columns=['deal_type_clean'])

def y_to_bin(dataf, y='dealstatus'):
    
    dataf[f'{y}_new'] = 0
    dataf.loc[lambda x: x[y]=='Completed', f'{y}_new'] = 1
    dataf[y] = dataf[f'{y}_new']
    return dataf.drop(columns=[f'{y}_new'])

In [3]:
dfraw = pd.read_excel('input/Data_set_NAFTA_EU_acquiror only.xls', sheet_name='Results')

In [4]:
dfclean = (dfraw.pipe(start_pipeline)
           .pipe(select_initial_columns)
           .pipe(select_initial_rows)
           .pipe(delete_na)
           .pipe(feature_generation)
           .pipe(clean_sector)
           .pipe(feature_selection)
          )

columns = [
    'dealstatus',
    'same_sector',
    'same_country',
    'target_sector',
    'acquiror_sector',
    'deal_type',
    'roa_target',
    'initial_stake',
]

dummy_columns = [
    'target_sector',
    'acquiror_sector',
    'deal_type'
]

dfprepped = (dfclean.pipe(filter_out_empty_rows, columns)
             .pipe(select_columns, columns)
             .pipe(clean_dealtype)
             .pipe(remove_roa_outliers)
             .pipe(transform_low_frequent_sectors, 'Other')
             .pipe(y_to_bin)
            )

dfprepped_featured = (dfclean.pipe(filter_out_empty_rows, columns)
             .pipe(select_columns, columns)
             .pipe(clean_dealtype)
             .pipe(remove_roa_outliers)
             .pipe(transform_low_frequent_sectors, 'Other')
             .pipe(define_dummies, dummy_columns)
             .pipe(y_to_bin)
            )

dfprepped.to_csv('input/analysis_features_dataset.csv', index=False)
dfprepped_featured.to_csv('input/final_featured_dataset.csv', index=False)

<ipython-input-2-3e2b9bfe4508>:42: FutureWarning: The default value of regex will change from True to False in a future version.
  dataf[c] = dataf[c].str.replace('n.a.', np.nan)


In [5]:
print('-- column completeness beforing checking:')
report_empty_rows(dfclean)

-- column completeness beforing checking:
dealstatus: 1.0
same_sector: 1.0
same_country: 1.0
target_sector: 0.97
acquiror_sector: 0.97
deal_type: 1.0
relative_size: 0.16
acquiror_size: 0.24
roa_target: 0.72
bid_premium: 0.29
initial_stake: 1.0
acquiror_free_cashflow: 0.34


In [6]:
print(f'n rows before filtering on completeness & outliers: {len(dfclean)}')
print(f'n rows after filtering on completeness & outliers: {len(dfprepped_featured)}')

n rows before filtering on completeness & outliers: 885
n rows after filtering on completeness & outliers: 564
